<a href="https://colab.research.google.com/github/kiruthika514/vgg16/blob/main/efficientnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow
from tensorflow import keras
from PIL import Image
import os
import string
import pickle
from pickle import dump, load
from tensorflow.keras.applications.efficientnet import EfficientNetB0  # Using EfficientNet
from tensorflow.keras.applications.efficientnet import preprocess_input
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer  # for text tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense  # Keras to build our CNN and LSTM
from tensorflow.keras.layers import LSTM, Embedding, Dropout

# EfficientNet replaces VGG16 for feature extraction
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input


In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# EfficientNet-specific preprocessing
from tensorflow.keras.applications.efficientnet import preprocess_input


Load Directories and test ,train split

In [ ]:
from pathlib import Path
import shutil
import os


In [ ]:
dataset_train = '/content/drive/MyDrive/remoteBLIP/train'
dataset_valid = '/content/drive/MyDrive/remoteBLIP/valid'
dataset_test = '/content/drive/MyDrive/remoteBLIP/test'
# os.mkdir(dataset_train)
# os.mkdir(dataset_test)


In [ ]:
import os

dataset_train = '/content/drive/MyDrive/remoteBLIP/train'
dataset_valid = '/content/drive/MyDrive/remoteBLIP/valid'
dataset_test = '/content/drive/MyDrive/remoteBLIP/test'

# Get the absolute path of the current working directory
current_directory = os.getcwd()

print("Current working directory:", current_directory)
print("Train dataset path:", os.path.abspath(dataset_train))
print("Valid dataset path:", os.path.abspath(dataset_valid))
print("Test dataset path:", os.path.abspath(dataset_test))


Current working directory: /content
Train dataset path: /content/drive/MyDrive/remoteBLIP/train
Valid dataset path: /content/drive/MyDrive/remoteBLIP/valid
Test dataset path: /content/drive/MyDrive/remoteBLIP/test


In [ ]:
print("Train images:",len(os.listdir(dataset_train)))
print("Validation images:",len(os.listdir(dataset_valid)))
print("Test images:",len(os.listdir(dataset_test)))

Train images: 8873
Validation images: 1094
Test images: 1126


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/remoteBLIP/train.csv")

# Convert file paths to dictionary keys and captions to values
train_mapping = {}

for _, row in df.iterrows():
    image_id = row["filepath"]  # Correct column for image filename
    captions = eval(row["captions"])  # Convert string list to actual list
    train_mapping[image_id] = captions

print(len(train_mapping))  # Number of unique images


8734


In [ ]:
import pandas as pd

# Load the CSV file
df_valid = pd.read_csv("/content/drive/MyDrive/remoteBLIP/valid.csv")

# Convert file paths to dictionary keys and captions to values
valid_mapping = {}

for _, row in df_valid.iterrows():
    image_id = row["filepath"]  # Use correct column for image filename
    captions = eval(row["captions"])  # Convert string list to actual list
    valid_mapping[image_id] = captions

print("Number of validation images with captions:", len(valid_mapping))  # Number of unique images


Number of validation images with captions: 1094


In [ ]:
import pandas as pd

# Load the CSV file
df_test = pd.read_csv("/content/drive/MyDrive/remoteBLIP/test.csv")

# Convert file paths to dictionary keys and captions to values
test_mapping = {}

for _, row in df_test.iterrows():
    image_id = row["filepath"]  # Use correct column for image filename
    captions = eval(row["captions"])  # Convert string list to actual list
    test_mapping[image_id] = captions

print("Number of test images with captions:", len(test_mapping))  # Number of unique images


Number of test images with captions: 1093


In [ ]:
import re

def clean(mapping):
    for key, captions in mapping.items():
        for i in range(len(captions)):
            # Take one caption at a time
            caption = captions[i]

            # Convert to lowercase
            caption = caption.lower()

            # Remove newlines correctly
            caption = caption.replace('\n', '')

            # Remove special characters & digits
            caption = re.sub(r'[^a-zA-Z ]', '', caption)

            # Remove extra spaces
            caption = re.sub(r'\s+', ' ', caption).strip()

            # Add start and end tokens
            caption = f'startseq {caption} endseq'

            captions[i] = caption  # Update the cleaned caption

    return mapping  # Return cleaned captions


In [ ]:
train_mapping = clean(train_mapping)
valid_mapping = clean(valid_mapping)
test_mapping = clean(test_mapping)

print(len(train_mapping))
print(len(valid_mapping))
print(len(test_mapping))


8734
1094
1093


In [ ]:
all_captions = []
for key in train_mapping:
    for caption in train_mapping[key]:
        all_captions.append(caption)
for key in valid_mapping:
    for caption in valid_mapping[key]:
        all_captions.append(caption)



In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Collect all captions from train, valid, and test mappings
all_captions = []

for mapping in [train_mapping, valid_mapping, test_mapping]:
    for captions in mapping.values():
        all_captions.extend(captions)  # Append all captions to a single list

# Initialize and fit tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)

# Get vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding token
print("Vocabulary Size:", vocab_size)


Vocabulary Size: 6530


extra checking

In [ ]:
import pandas as pd

df_train = pd.read_csv("/content/drive/MyDrive/remoteBLIP/train.csv")

# Show 5 random captions to check if they look normal
print(df_train.sample(5))


                                               captions  \
723   ['White waves of green ocean are near the yell...   
7884  ['the ease of the road is surrounded by land a...   
7601  ['several ships were parked at sea.'\n 'many w...   
7900  ['the ease of the road are surrounded by dark ...   
8542  ['a simple playground is situated in the gap b...   

                        filepath  
723          train/beach_130.jpg  
7884       train/viaduct_346.jpg  
7601  train/storagetanks_343.jpg  
7900       train/viaduct_360.jpg  
8542             train/00597.jpg  


In [ ]:
import pandas as pd

# Load CSV
df_train = pd.read_csv("/content/drive/MyDrive/remoteBLIP/train.csv")

# Convert 'captions' column from string to list
df_train["captions"] = df_train["captions"].apply(eval)  # Converts '["caption1", "caption2"]' → ["caption1", "caption2"]

# Select ONLY the first caption for each image
train_mapping = {row["filepath"]: row["captions"][0] for _, row in df_train.iterrows()}

print("Number of unique training images:", len(train_mapping))

# Verify a few samples
for img, caption in list(train_mapping.items())[:5]:
    print(f"{img}: {caption}")


Number of unique training images: 8734
train/airport_1.jpg: Many aircraft are parked next to a long building in an airport.Many planes are parked next to a long building at an airport.Many planes are parked next to a long building in an airport.many planes are parked next to a long building at an airport.many planes are parked next to a long building in an airport .
train/airport_10.jpg: some planes are parked in an airport.the airport here is full of planes and containers.the airport here is full of airplanes and containers .Some planes are parked at an airport.some planes are parked in an airport .
train/airport_100.jpg: Many aircraft are parked in an airport near many runways.There are a lot of the same size planes in the airport.there are many planes with same size in the airport .Many planes are parked at an airport near many runways.many planes are parked in an airport near many runways .
train/airport_101.jpg: Many aircraft are parked near a large building on an airport.Many pla

In [ ]:
# Print the 5 longest captions in all_captions
longest_captions = sorted(all_captions, key=lambda x: len(x.split()), reverse=True)[:5]

print("Top 5 Longest Captions:")
for i, caption in enumerate(longest_captions, 1):
    print(f"{i}. ({len(caption.split())} words) {caption}")


Top 5 Longest Captions:
1. (119 words) startseq a road with cars and a large piece of green meadow are on both sides of an irregular green pond surrounded by some treesa road with cars and a large piece of green meadow are located on two sides of an irregular green pond surrounded by some treesa road with car and a large piece of green lawn are on two sides of an irregular green pond surrounded by some treesa road with cars and a large piece of green meadow are on two sides of an irregular green lake surrounded by some treesa road with cars and a large piece of green meadow are in two sides of an irregular green pond surrounded by some trees endseq
2. (113 words) startseq a bridge is on a river with several pieces of agricultural land and several buildings with a pond on both sides of ita bridge is located on a river with several pieces of agricultural land and several buildings with a pond on two sides of ita bridge is on a river with several pieces of agricultural land and several bu

actual one

In [ ]:
if all_captions:
    max_length = max(len(caption.split()) for caption in all_captions)
else:
    max_length = 0  # Set a default value if no captions exist

print("Maximum Caption Length:", max_length)


Maximum Caption Length: 119


Feature Extraction

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import preprocess_input
import numpy as np
import os
from PIL import Image

def extract_features(directory):
    model = EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
    features = {}

    for pic in os.listdir(directory):
        file = os.path.join(directory, pic)

        # Load and preprocess image
        image = Image.open(file).convert("RGB")
        image = image.resize((224, 224))
        image = np.array(image)
        image = np.expand_dims(image, axis=0)
        image = preprocess_input(image)

        print(f"Processing: {file}")

        # Extract features
        feature = model.predict(image)
        features[file] = feature

    return features


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm  # For progress bar
import pickle  # For saving features

# Load EfficientNet model (lighter and faster than VGG16)
base_model = EfficientNetB0(weights="imagenet")
model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)  # Second-last layer

def extract_features(directory, batch_size=32):
    features = {}
    image_paths = [os.path.join(directory, img) for img in os.listdir(directory)]

    for i in tqdm(range(0, len(image_paths), batch_size), desc="Extracting Features"):
        batch_paths = image_paths[i : i + batch_size]
        batch_images = []

        for file in batch_paths:
            img = load_img(file, target_size=(224, 224))
            img = img_to_array(img)
            img = preprocess_input(img)  # Normalize for EfficientNet
            batch_images.append(img)

        batch_images = np.array(batch_images)
        batch_features = model.predict(batch_images)  # Predict in batches

        for j, file in enumerate(batch_paths):
            features[file] = batch_features[j]

    return features

# Extract and save features (train, validation, test)
train_features = extract_features(dataset_train)
valid_features = extract_features(dataset_valid)
test_features = extract_features(dataset_test)

# Save extracted features for later use (avoids re-running)
with open("train_features.pkl", "wb") as f:
    pickle.dump(train_features, f)

with open("valid_features.pkl", "wb") as f:
    pickle.dump(valid_features, f)

with open("test_features.pkl", "wb") as f:
    pickle.dump(test_features, f)


Extracting Features:   0%|          | 0/278 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


Extracting Features:   0%|          | 1/278 [00:05<24:41,  5.35s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


Extracting Features:   1%|          | 2/278 [00:10<24:34,  5.34s/it]

...

Extracting Features:  94%|█████████▍| 34/36 [02:23<00:07,  3.60s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


Extracting Features:  97%|█████████▋| 35/36 [02:26<00:03,  3.34s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step


Extracting Features: 100%|██████████| 36/36 [02:26<00:00,  4.08s/it]


In [ ]:
from pickle import dump

# Save extracted features for EfficientNet
dump(train_features, open('train_features_EfficientNet.p', "wb"))
dump(valid_features, open('valid_features_EfficientNet.p', "wb"))
dump(test_features, open('test_features_EfficientNet.p', "wb"))


In [ ]:
list(train_mapping.items())[:5]  # Shows the first 5 image-caption pairs



[('train/airport_1.jpg',
  'Many aircraft are parked next to a long building in an airport.Many planes are parked next to a long building at an airport.Many planes are parked next to a long building in an airport.many planes are parked next to a long building at an airport.many planes are parked next to a long building in an airport .'),
 ('train/airport_10.jpg',
  'some planes are parked in an airport.the airport here is full of planes and containers.the airport here is full of airplanes and containers .Some planes are parked at an airport.some planes are parked in an airport .'),
 ('train/airport_100.jpg',
  'Many aircraft are parked in an airport near many runways.There are a lot of the same size planes in the airport.there are many planes with same size in the airport .Many planes are parked at an airport near many runways.many planes are parked in an airport near many runways .'),
 ('train/airport_101.jpg',
  'Many aircraft are parked near a large building on an airport.Many plane

In [ ]:
df['captions'] = df['captions'].apply(lambda x: x.split('. '))


In [ ]:
print(df.columns)


Index(['captions', 'filepath'], dtype='object')


In [ ]:
print(df.head())  # Check the first few rows
print(df.dtypes)  # Check data types
print(type(df['captions'].iloc[0]))  # Check type of a single caption entry


                                            captions               filepath
0  [['Many aircraft are parked next to a long bui...    train/airport_1.jpg
1  [['some planes are parked in an airport.'\n 't...   train/airport_10.jpg
2  [['Many aircraft are parked in an airport near...  train/airport_100.jpg
3  [['Many aircraft are parked near a large build...  train/airport_101.jpg
4  [['several buildings and green trees are aroun...  train/airport_102.jpg
captions    object
filepath    object
dtype: object
<class 'list'>


In [ ]:
df['captions'] = df['captions'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)


In [ ]:
print(df['captions'].iloc[0])  # Should now print a flat list
print(type(df['captions'].iloc[0]))  # Should be <class 'list'>, NOT nested


['Many aircraft are parked next to a long building in an airport.'
 'Many planes are parked next to a long building at an airport.'
 'Many planes are parked next to a long building in an airport.'
 'many planes are parked next to a long building at an airport.'
 'many planes are parked next to a long building in an airport .']
<class 'str'>


In [ ]:
train_mapping = {}

for image, captions in zip(df['filepath'], df['captions']):
    train_mapping[image] = captions  # Directly assign the fixed list

# Verify Output
for i, (key, value) in enumerate(train_mapping.items()):
    print(f"{i+1}. Image: {key}\n   Captions: {value}\n")
    if i == 4:  # Print only the first 5 entries
        break


1. Image: train/airport_1.jpg
   Captions: ['Many aircraft are parked next to a long building in an airport.'
 'Many planes are parked next to a long building at an airport.'
 'Many planes are parked next to a long building in an airport.'
 'many planes are parked next to a long building at an airport.'
 'many planes are parked next to a long building in an airport .']

2. Image: train/airport_10.jpg
   Captions: ['some planes are parked in an airport.'
 'the airport here is full of planes and containers.'
 'the airport here is full of airplanes and containers .'
 'Some planes are parked at an airport.'
 'some planes are parked in an airport .']

3. Image: train/airport_100.jpg
   Captions: ['Many aircraft are parked in an airport near many runways.'
 'There are a lot of the same size planes in the airport.'
 'there are many planes with same size in the airport .'
 'Many planes are parked at an airport near many runways.'
 'many planes are parked in an airport near many runways .']

4.

In [ ]:
import ast

df['captions'] = df['captions'].apply(ast.literal_eval)  # Convert string lists to actual lists


In [ ]:
train_mapping

{'train/airport_1.jpg': "['Many aircraft are parked next to a long building in an airport.'\n 'Many planes are parked next to a long building at an airport.'\n 'Many planes are parked next to a long building in an airport.'\n 'many planes are parked next to a long building at an airport.'\n 'many planes are parked next to a long building in an airport .']",
 'train/airport_10.jpg': "['some planes are parked in an airport.'\n 'the airport here is full of planes and containers.'\n 'the airport here is full of airplanes and containers .'\n 'Some planes are parked at an airport.'\n 'some planes are parked in an airport .']",
 'train/airport_100.jpg': "['Many aircraft are parked in an airport near many runways.'\n 'There are a lot of the same size planes in the airport.'\n 'there are many planes with same size in the airport .'\n 'Many planes are parked at an airport near many runways.'\n 'many planes are parked in an airport near many runways .']",
 'train/airport_101.jpg': "['Many aircraf

In [ ]:
for img, captions in train_mapping.items():
    print(f"Image: {img}")
    print(f"Captions: {captions}")  # Should be a list, not a string
    print(f"Type of captions: {type(captions)}")  # Should be <class 'list'>
    break  # Print only one sample


Image: train/airport_1.jpg
Captions: ['Many aircraft are parked next to a long building in an airport.'
 'Many planes are parked next to a long building at an airport.'
 'Many planes are parked next to a long building in an airport.'
 'many planes are parked next to a long building at an airport.'
 'many planes are parked next to a long building in an airport .']
Type of captions: <class 'str'>


In [ ]:
train_mapping = {}

for image, captions in zip(df['filepath'], df['captions']):
    # Each 'captions' is a list containing a single long string
    if isinstance(captions, list) and len(captions) == 1 and isinstance(captions[0], str):
        captions = captions[0].split(".")  # Split at sentence boundaries

    # Remove empty strings and strip spaces
    captions = [cap.strip() for cap in captions if cap.strip()]

    # Add startseq and endseq
    train_mapping[image] = [f"startseq {caption} endseq" for caption in captions]

# Verify Output
for i, (key, value) in enumerate(train_mapping.items()):
    print(f"{i+1}. Image: {key}\n   Captions: {value}\n")
    if i == 4:  # Print only first 5 entries
        break


1. Image: train/airport_1.jpg
   Captions: ['startseq Many aircraft are parked next to a long building in an airport endseq', 'startseq Many planes are parked next to a long building at an airport endseq', 'startseq Many planes are parked next to a long building in an airport endseq', 'startseq many planes are parked next to a long building at an airport endseq', 'startseq many planes are parked next to a long building in an airport endseq']

2. Image: train/airport_10.jpg
   Captions: ['startseq some planes are parked in an airport endseq', 'startseq the airport here is full of planes and containers endseq', 'startseq the airport here is full of airplanes and containers endseq', 'startseq Some planes are parked at an airport endseq', 'startseq some planes are parked in an airport endseq']

3. Image: train/airport_100.jpg
   Captions: ['startseq Many aircraft are parked in an airport near many runways endseq', 'startseq There are a lot of the same size planes in the airport endseq', 's

In [ ]:
train_features = load(open("./train_features_EfficientNet.p", "rb"))
valid_features = load(open("./valid_features_EfficientNet.p", "rb"))
test_features = load(open("./test_features_EfficientNet.p", "rb"))


Loading captions

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, Embedding, LSTM

# Update feature vector shape for EfficientNetB0
inputs1 = Input(shape=(1280,))  # Change from 4096 (VGG) to 1280 (EfficientNet)
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# LSTM sequence model
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

# Merging both models
decoder1 = fe2 + se3
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

# Define model
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Summarize model
print(model.summary())


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 119)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 119, 256)       │      1,671,680 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 1280)           │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 119, 256)       │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, 119)            │              0 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 256)            │        327,936 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 256)            │        525,312 │ dropout_1[0][0],       │
│                           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 256)            │              0 │ dense[0][0],           │
│                           │                        │                │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 256)            │         65,792 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 6530)           │      1,678,210 │ dense_1[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,268,930 (16.28 MB)

 Trainable params: 4,268,930 (16.28 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    X1, X2, y = list(), list(), list()
    n = 0
    while 1:
        for key in data_keys:
            n += 1
            map_key = key.replace("./", "")  # Ensure key matches feature dictionary
            captions = mapping[map_key]  # Get captions for the image

            for caption in captions:
                # Convert caption to sequence
                seq = tokenizer.texts_to_sequences([caption])[0]

                # Create input-output pairs
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]

                    # Pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]

                    # One-hot encode output word
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                    # Store sequences
                    X1.append(features[key])  # Use the correct feature vector
                    X2.append(in_seq)
                    y.append(out_seq)

            # Yield batch data
            if n == batch_size:
                yield (np.array(X1), np.array(X2)), np.array(y)
                X1, X2, y = list(), list(), list()  # Reset batch lists
                n = 0


In [ ]:
print(list(train_mapping.keys())[:5])  # Print first 5 keys


['train/airport_1.jpg', 'train/airport_10.jpg', 'train/airport_100.jpg', 'train/airport_101.jpg', 'train/airport_102.jpg']


In [ ]:
map_key = key.replace("./", "")  # This correctly formats the key


In [ ]:
print(map_key in train_mapping)  # Should return True if the mapping exists
print(train_mapping[map_key])  # Should return the captions


True
['startseq several buildings and green trees are around a piece of bare land endseq', 'startseq some planes are parked near an airport with parking endseq', 'startseq some planes are parked near an airport with parking lot endseq', 'startseq several green buildings and trees are around a piece of bare land endseq', 'startseq several buildings and green trees are around a piece of bareland endseq']


In [ ]:
print(type(train_mapping[map_key]))  # Should be <class 'list'>
print(train_mapping[map_key])  # Should print a list of separate captions


<class 'list'>
['startseq several buildings and green trees are around a piece of bare land endseq', 'startseq some planes are parked near an airport with parking endseq', 'startseq some planes are parked near an airport with parking lot endseq', 'startseq several green buildings and trees are around a piece of bare land endseq', 'startseq several buildings and green trees are around a piece of bareland endseq']


In [ ]:
for image, captions in zip(df['filepath'], df['captions']):
    train_mapping[image] = captions  # Check if this is storing a proper list


In [ ]:
for image, captions in zip(df['filepath'], df['captions']):
    train_mapping[image] = [cap.strip() + " ." for cap in captions[0].split('.') if cap]  # Ensure proper sentence separation


In [ ]:
print(train_mapping['train/airport_102.jpg'])


['several buildings and green trees are around a piece of bare land .', 'some planes are parked near an airport with parking .', 'some planes are parked near an airport with parking lot .', 'several green buildings and trees are around a piece of bare land .', 'several buildings and green trees are around a piece of bareland .']


In [ ]:
for image, captions in zip(df['filepath'], df['captions']):
    train_mapping[image] = ["startseq " + cap.strip() + " endseq" for cap in captions[0].split('.') if cap]


In [ ]:
print(train_mapping['train/airport_102.jpg'])


['startseq several buildings and green trees are around a piece of bare land endseq', 'startseq some planes are parked near an airport with parking endseq', 'startseq some planes are parked near an airport with parking lot endseq', 'startseq several green buildings and trees are around a piece of bare land endseq', 'startseq several buildings and green trees are around a piece of bareland endseq']


In [ ]:
key = './train/00001.jpg'
map_key = key.replace("./","")
train_mapping[map_key]

['startseq a football field with several buildiings surroudned endseq',
 'startseq a rectangular playground and many tall buildings surrounded endseq',
 'startseq many buildings and green trees are around a playground endseq',
 'startseq many buildings are in different blocks with many green trees and a playground endseq',
 'startseq a playground is surrounded by many trees and buildings endseq']

In [ ]:
train_filenames =  os.listdir('/content/drive/MyDrive/remoteBLIP/train')
valid_filenames =  os.listdir('/content/drive/MyDrive/remoteBLIP/valid')
test_filenames = os.listdir('/content/drive/MyDrive/remoteBLIP/test')

In [ ]:
mystring = '/content/drive/MyDrive/remoteBLIP/train'
train_filenames = [mystring + s for s in train_filenames]
mystring = '/content/drive/MyDrive/remoteBLIP/valid'
valid_filenames = [mystring + s for s in valid_filenames]
mystring = '/content/drive/MyDrive/remoteBLIP/test'
test_filenames = [mystring + s for s in test_filenames]

In [ ]:
# Create a reverse mapping (index -> word) once
index_to_word = {index: word for word, index in tokenizer.word_index.items()}

# Function to get word from index
def idx_to_word(integer, index_to_word):
    return index_to_word.get(integer, None)  # Direct lookup


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

def generate_caption(image_name, model, test_mapping, test_features, tokenizer, max_length):
    # Load the image
    img_path = './test/' + image_name
    try:
        image = Image.open(img_path)
    except FileNotFoundError:
        print(f"Error: Image file '{img_path}' not found.")
        return

    # Get actual captions from test_mapping
    key = 'test/' + image_name
    captions = test_mapping.get(key, [])

    if not captions:
        print(f"Warning: No captions found for {image_name}.")
    else:
        print('---------------------Actual---------------------')
        for caption in captions:
            print(caption)

    # Predict the caption
    try:
        y_pred = predict_caption(model, test_features.get(img_path, None), tokenizer, max_length)
    except KeyError:
        print(f"Error: No feature found for {img_path}.")
        return

    print('--------------------Predicted--------------------')
    print(y_pred)

    # Display the image
    plt.imshow(image)
    plt.axis('off')  # Hide axes for a cleaner look
    plt.show()  # Ensure the image is displayed


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

def predict_caption(model, image, tokenizer, max_length, idx_to_word):
    # Initialize the sequence with start token
    in_text = 'startseq'

    # Prepare the input sequence and pad it outside the loop to avoid repetitive padding
    sequence = tokenizer.texts_to_sequences([in_text])[0]
    sequence = pad_sequences([sequence], maxlen=max_length)

    # Iterate over the max length of sequence
    for i in range(max_length):
        # Predict next word based on image and current sequence
        yhat = model.predict([image, sequence], verbose=0)
        yhat = np.argmax(yhat)  # Get index with the highest probability

        # Convert index to word using the provided function
        word = idx_to_word(yhat, tokenizer)

        # If word is None (no valid prediction), stop the loop
        if word is None:
            break

        # Append the word to the sequence for the next prediction
        in_text += " " + word

        # Clean up the sequence, remove startseq, endseq, and trim whitespace
        in_text = in_text.replace("startseq", "").replace("endseq", "").strip()

        # Stop if 'endseq' is predicted
        if word == 'endseq':
            break

        # Update the sequence with the new word
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)

    return in_text


def load_caption_data(csv_path):
    # Assuming the CSV contains image file names and their corresponding actual captions
    df = pd.read_csv(csv_path)
    # Assuming the format: 'image_name', 'caption'
    # Adjust based on your CSV structure
    image_caption_data = {}

    for idx, row in df.iterrows():
        image_caption_data[row['image_name']] = row['caption'].split(',')  # Assuming multiple captions per image, separated by commas

    return image_caption_data


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

# Validate with test data
def PREDICT_BLEU():
    actual, predicted = list(), list()

    for key in test_filenames:
        map_key = key.replace("./", "")
        captions = test_mapping.get(map_key, [])
        # Ensure proper caption formatting by removing 'startseq' and 'endseq'
        captions = [caption.replace('startseq', '').replace('endseq', '').strip() for caption in captions]

        # Predict the caption for image
        y_pred = predict_caption(model, test_features[key], tokenizer, max_length)

        # Split the predicted caption into words and clean it
        predicted_words = y_pred.split()  # Split predicted caption into words

        # Split actual captions into words
        actual_captions = [caption.split() for caption in captions]

        # Append to the list
        actual.append(actual_captions)
        predicted.append(predicted_words)  # Use words instead of slicing the string

    # Calculate BLEU scores
    print("BLEU-1: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print("BLEU-2: %f" % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print("BLEU-3: %f" % corpus_bleu(actual, predicted, weights=(0.33, 0.33, 0.33, 0)))
    print("BLEU-4: %f" % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))


In [ ]:
epochs = 20
batch_size = 32
steps = len(valid_filenames) // batch_size
fix=[25,50,75,100]

In [ ]:
import os

# Assuming 'train' and 'valid' directories contain your image files
train_dir = '/content/drive/MyDrive/remoteBLIP/train'
valid_dir = '/content/drive/MyDrive/remoteBLIP/valid'

# List filenames in the train and validation directories
train_filenames = os.listdir(train_dir)
valid_filenames = os.listdir(valid_dir)

# Print the number of files
print(f'Train files: {len(train_filenames)}')
print(f'Valid files: {len(valid_filenames)}')


Train files: 8873
Valid files: 1094


In [ ]:
# Filter train_filenames to only include mapped images
filtered_train_filenames = [key for key in train_filenames if key.replace("./", "") in train_mapping]
filtered_valid_filenames = [key for key in valid_filenames if key.replace("./", "") in valid_mapping]

print(f"Total images before filtering: {len(train_filenames)}")
print(f"Total images after filtering: {len(filtered_train_filenames)}")


Total images before filtering: 8873
Total images after filtering: 0


In [ ]:
filtered_train_filenames = [f for f in train_filenames if f in train_mapping]
filtered_valid_filenames = [f for f in valid_filenames if f in valid_mapping]


In [ ]:
train_generator = data_generator(filtered_train_filenames, train_mapping, train_features, tokenizer, max_length, vocab_size, batch_size)
valid_generator = data_generator(filtered_valid_filenames, valid_mapping, valid_features, tokenizer, max_length, vocab_size, batch_size)


In [ ]:
for i, batch in enumerate(train_generator):
    print(f"Batch {i}: {batch}")
    if i == 2:  # Stop after 2 batches
        break


Batch 0: (array([], dtype=float64), array([], dtype=float64))
Batch 1: (array([], dtype=float64), array([], dtype=float64))
Batch 2: (array([], dtype=float64), array([], dtype=float64))


In [ ]:
# Print a few sample keys from each list
print("Sample train_filenames:", train_filenames[:5])
print("Sample train_mapping keys:", list(train_mapping.keys())[:5])


Sample train_filenames: ['square_212.jpg', 'square_21.jpg', 'square_206.jpg', 'square_243.jpg', 'square_223.jpg']
Sample train_mapping keys: ['train/airport_1.jpg', 'train/airport_10.jpg', 'train/airport_100.jpg', 'train/airport_101.jpg', 'train/airport_102.jpg']


In [ ]:
train_generator = data_generator(train_filenames, train_mapping, train_features, tokenizer, max_length, vocab_size, batch_size)

# Get one batch from generator
(X1, X2), y = next(train_generator)

print("X1 shape (Image Features):", X1.shape)  # Should be (batch_size, feature_dim)
print("X2 shape (Text Sequences):", X2.shape)  # Should be (batch_size, max_length)
print("y shape (Output Labels):", y.shape)    # Should be (batch_size, vocab_size)


Key square_212.jpg not found in mapping!
Key square_21.jpg not found in mapping!
Key square_206.jpg not found in mapping!
Key square_243.jpg not found in mapping!
Key square_223.jpg not found in mapping!
Key square_241.jpg not found in mapping!
Key square_226.jpg not found in mapping!
Key square_24.jpg not found in mapping!
Key square_236.jpg not found in mapping!
Key square_222.jpg not found in mapping!
Key square_227.jpg not found in mapping!
Key square_23.jpg not found in mapping!
Key square_238.jpg not found in mapping!
Key square_231.jpg not found in mapping!
Key square_229.jpg not found in mapping!
Key square_235.jpg not found in mapping!
Key square_221.jpg not found in mapping!
Key square_242.jpg not found in mapping!
Key square_228.jpg not found in mapping!
Key square_233.jpg not found in mapping!
Key square_240.jpg not found in mapping!
Key square_232.jpg not found in mapping!
Key square_234.jpg not found in mapping!
Key square_237.jpg not found in mapping!
Key square_239.jpg 

ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
train_generator = data_generator(train_filenames, train_mapping, train_features, tokenizer, max_length, vocab_size, batch_size)
valid_generator = data_generator(valid_filenames, valid_mapping, valid_features, tokenizer, max_length, vocab_size, batch_size)
steps = len(train_filenames) // batch_size
validation_steps = len(valid_filenames) // batch_size

model.fit(train_generator, epochs=25, steps_per_epoch=steps, validation_data=valid_generator,validation_steps=validation_steps,verbose=1)


Key square_212.jpg not found in mapping!
Key square_21.jpg not found in mapping!
Key square_206.jpg not found in mapping!
Key square_243.jpg not found in mapping!
Key square_223.jpg not found in mapping!
Key square_241.jpg not found in mapping!
Key square_226.jpg not found in mapping!
Key square_24.jpg not found in mapping!
Key square_236.jpg not found in mapping!
Key square_222.jpg not found in mapping!
Key square_227.jpg not found in mapping!
Key square_23.jpg not found in mapping!
Key square_238.jpg not found in mapping!
Key square_231.jpg not found in mapping!
Key square_229.jpg not found in mapping!
Key square_235.jpg not found in mapping!
Key square_221.jpg not found in mapping!
Key square_242.jpg not found in mapping!
Key square_228.jpg not found in mapping!
Key square_233.jpg not found in mapping!
Key square_240.jpg not found in mapping!
Key square_232.jpg not found in mapping!
Key square_234.jpg not found in mapping!
Key square_237.jpg not found in mapping!
Key square_239.jpg 

ValueError: Layer "functional_2" expects 2 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor 'data:0' shape=(None,) dtype=float64>]

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    X1, X2, y = list(), list(), list()
    n = 0
    while True:  # Infinite loop for generator
        for key in data_keys:
            map_key = 'train/' + key.replace("./", "")
            if map_key in mapping:
                captions = mapping[map_key]
                for caption in captions:
                    seq = tokenizer.texts_to_sequences([caption])[0]
                    for i in range(1, len(seq)):
                        in_seq, out_seq = seq[:i], seq[i]
                        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                        X1.append(features[key][0])
                        X2.append(in_seq)
                        y.append(out_seq)
                n += 1
                if n == batch_size:
                    X1, X2, y = np.array(X1), np.array(X2), np.array(y)
                    yield (X1, X2), y
                    X1, X2, y = list(), list(), list()
                    n = 0
            else:
                print(f"Key {map_key} not found in mapping")

In [ ]:
import numpy as np

def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    while True:
        n = 0
        X, y = list(), list()

        # Loop through the filenames in data_keys (train_filenames, valid_filenames)
        for key in data_keys:
            map_key = key.replace("./", "")  # Ensure key matches feature dictionary

            # Ensure the key exists in mapping
            if map_key not in mapping:
                print(f"Key {map_key} not found in mapping!")
                continue  # Skip if the key doesn't exist

            captions = mapping[map_key]

            # Extract image features (assuming 'features' is a pre-extracted features dictionary)
            img_features = features.get(key)  # features should be a dictionary of image features
            if img_features is None:
                print(f"Image feature for {key} not found!")
                continue

            # Generate sequence for the captions
            for caption in captions:
                seq = tokenizer.texts_to_sequences([caption])[0]
                seq = pad_sequences([seq], maxlen=max_length)[0]

                # Append to batch data
                X.append(img_features)
                y.append(seq)

                n += 1
                if n >= batch_size:
                    break  # Return batch after accumulating 'batch_size' samples

        # Convert lists to arrays before yielding
        yield np.array(X), np.array(y)


In [ ]:
train_generator = data_generator(train_filenames, train_mapping, train_features, tokenizer, max_length, vocab_size, batch_size)
valid_generator = data_generator(valid_filenames, valid_mapping, valid_features, tokenizer, max_length, vocab_size, batch_size)
steps = len(train_filenames) // batch_size
validation_steps = len(valid_filenames) // batch_size

model.fit(train_generator, epochs=25, steps_per_epoch=steps, validation_data=valid_generator, validation_steps=validation_steps, verbose=1)

Key square_212.jpg not found in mapping!
Key square_21.jpg not found in mapping!
Key square_206.jpg not found in mapping!
Key square_243.jpg not found in mapping!
Key square_223.jpg not found in mapping!
Key square_241.jpg not found in mapping!
Key square_226.jpg not found in mapping!
Key square_24.jpg not found in mapping!
Key square_236.jpg not found in mapping!
Key square_222.jpg not found in mapping!
Key square_227.jpg not found in mapping!
Key square_23.jpg not found in mapping!
Key square_238.jpg not found in mapping!
Key square_231.jpg not found in mapping!
Key square_229.jpg not found in mapping!
Key square_235.jpg not found in mapping!
Key square_221.jpg not found in mapping!
Key square_242.jpg not found in mapping!
Key square_228.jpg not found in mapping!
Key square_233.jpg not found in mapping!
Key square_240.jpg not found in mapping!
Key square_232.jpg not found in mapping!
Key square_234.jpg not found in mapping!
Key square_237.jpg not found in mapping!
Key square_239.jpg 

ValueError: Layer "functional_2" expects 2 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor 'data:0' shape=(None,) dtype=float64>]